In [ ]:
import numpy as np # linear algebra
from numpy.random import seed
from numpy.random import rand
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import datasets 
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense
from keras import losses, optimizers
from keras.utils import to_categorical


In [ ]:
print(np.add(np.array([[1,2,3], [1,2,3]]), 3*np.array([[1,2,3], [1,2,3]])))

In [104]:
class Particle:
    def __init__(self, weights, velocity, omega, c1, c2, lower_bound, upper_bound, particle_id):
        self.id = particle_id
        self.weights = weights
        self.best_weights = weights
        self.velocity = np.array(velocity)
        self.omega = omega
        self.c1 = c1
        self.c2 = c2
        self.lower_bound = lower_bound
        self.upper_bound = upper_bound
        self.best_valuation = 0
        self.history = []
        
    def update(self, best_particle):
        self.update_velocity(best_particle)
        self.check_velocity()
        self.update_particle()
        return self.weights
    
    def update_velocity(self, bp):
        # v = Wv + c1r1(pi - xi) + c2r2(g - xi)
        # v - ubrzanje, W - omega, c1 i c2 unapred zadati parametri
        # r1 i r2 slucajni brojevi iz (0,1) uniformne raspodele
        # pi - najbolje resenje trenutne jedinke
        # g - najbojle globalno resenje
        r1 = np.random.random()
        r2 = np.random.random()
        
        self.velocity = self.omega * self.velocity + self.c1*r1*(np.add(np.array(self.best_weights), (-1)*np.array(self.weights))) + self.c2*r2*(np.add(np.array(bp), (-1)*np.array(self.weights))) 
        
    def check_velocity(self):
        #TODO
        return True
    
    def update_particle(self):
        # xi = xi + vi
        self.weights = np.add(np.array(self.weights), self.velocity)
        
    def update_valuation(self, valuation):
        self.history.append(valuation)
        if(valuation > self.best_valuation):
            self.best_valuation = valuation
            self.best_weights = self.weights
            
    def get_weights(self):
        return self.weights
        

In [114]:
class PSO:
    def __init__(self, num_particles, num_iters, training_x, training_y, model, shapes, c1, c2, w, lower_bound, upper_bound):
        self.num_iters = num_iters
        self.training_x = training_x
        self.training_y = training_y
        self.model = model
        self.best_particle = self.make_velocity(shapes)
        self.best_evaluation = 0
        self.history = []
        self.particles = self.make_particles(num_particles, shapes, c1, c2, w, lower_bound, upper_bound)
        self.evaluate_particles()
        self.combo = 0
        
        
    def make_particles(self, size, shape, c1, c2, w, low, upp):
        particles = []
        
        velocity = self.make_velocity(shape)
        
        for i in range(size):
            weights = self.make_weights(shape, low, upp)
            particles.append(Particle(weights, velocity, w, c1, c2, low, upp, i))
            
        return particles
    
    def make_velocity(self, shapes):
        velocity = []
        
        for shape in shapes:
            velocity.append(np.zeros(shape))
        
        return velocity
                
        
    def make_weights(self, shapes, low, upp):
        weights = []
        
        for shape in shapes:
            if(len(shape) == 1):
                # wght da dobijemo vrednosti iz (0, upp-low)
                # rec da ih vrati na (low, upp)
                
                wght = rand(shape[0])*(upp-low)
                rec = np.full(shape, low)
                
                weights.append(np.add(wght, rec))
                
            else:
                wght = rand(shape[0], shape[1])*(upp-low)
                rec = np.full(shape, low)
                
                weights.append(np.add(wght, rec))
                
        return weights
        
        
    def evaluate_particles(self):
        
        for particle in self.particles:
            self.model.set_weights(particle.get_weights())
            test_loss, test_acc = self.model.evaluate(self.training_x, self.training_y, verbose = 0)
            
            particle.update_valuation(test_acc)
            print("Jedinka {} :  {}".format(particle.id, test_acc))
            if(test_acc > self.best_evaluation):
                print("Better solution: ", test_acc, '\n')
                self.best_particle = particle.get_weights()
                self.best_evaluation = test_acc
        
        print('-------------------------------------------------------------------------------')
        self.history.append(self.best_evaluation)
        
    def isclose(self, a, b, rel_tol=1e-09, abs_tol=0.0):
        return abs(a-b) <= max(rel_tol * max(abs(a), abs(b)), abs_tol)
    
    def update_particles(self):
        
        for particle in self.particles:
            particle.update(self.best_particle)
            
    def get_particles(self):
        return self.particles
    
    def start_pso(self):
        
        for i in range(self.num_iters):
            partly_best_solution = self.best_evaluation
            self.update_particles()
            self.evaluate_particles()
            
            print("Iteracija: {}\nPreciznost: {}\n".format(i+1, self.best_evaluation))
            
            if(self.isclose(partly_best_solution, self.best_evaluation, 0.001)):
                self.combo += 1
            else:
                self.combo = 0
            
                
        return self.best_particle
            
        

In [ ]:
np.add(rand(10, 2)*4, np.full())

In [ ]:
np.random.random()

In [115]:
data = datasets.load_iris()

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data.data, 
                                                    data.target, 
                                                    test_size=0.33)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

print(x_train.shape)




print('Pre skaliranja:\n', x_train[0])
scaler = preprocessing.StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)
print('\nPosle skaliranja:\n', x_train[0])

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)


model = Sequential()
model.add(Dense(units=100, input_dim=x_train.shape[1], activation='relu'))
model.add(Dense(units=y_train.shape[1], activation='sigmoid'))
model.summary()


shapes = [i.shape for i in model.get_weights()]

model.compile(optimizer='adam', 
              loss=losses.categorical_crossentropy, 
              metrics=['accuracy'])

print('Making PSO: ')
pso = PSO(20, 10, x_train, y_train, model, shapes, 0.5, 1.0, 0.3, -2, 2)
best_particle = pso.start_pso()

(100, 4)
Pre skaliranja:
 [5.  3.4 1.5 0.2]

Posle skaliranja:
 [-0.98668555  0.77579907 -1.23016506 -1.2398145 ]
(100, 4)
(50, 4)
(100, 3)
(50, 3)
Model: "sequential_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_53 (Dense)             (None, 100)               500       
_________________________________________________________________
dense_54 (Dense)             (None, 3)                 303       
Total params: 803
Trainable params: 803
Non-trainable params: 0
_________________________________________________________________
Making PSO: 
Jedinka 0 :  0.20000000298023224
Better solution:  0.20000000298023224 

Jedinka 1 :  0.12999999523162842
Jedinka 2 :  0.46000000834465027
Better solution:  0.46000000834465027 

Jedinka 3 :  0.4099999964237213
Jedinka 4 :  0.17000000178813934
Jedinka 5 :  0.4300000071525574
Jedinka 6 :  0.6100000143051147
Better solution:  0.6100000143051147 

Jedinka 7

Jedinka 8 :  0.6600000262260437
Jedinka 9 :  0.6600000262260437
Jedinka 10 :  0.3499999940395355
Jedinka 11 :  0.6700000166893005
Jedinka 12 :  0.6600000262260437
Jedinka 13 :  0.6700000166893005
Jedinka 14 :  0.6100000143051147
Jedinka 15 :  0.6700000166893005
Jedinka 16 :  0.6399999856948853
Jedinka 17 :  0.6399999856948853
Jedinka 18 :  0.6499999761581421
Jedinka 19 :  0.6700000166893005
-------------------------------------------------------------------------------
Iteracija: 10
Preciznost: 0.6700000166893005



In [116]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print(test_acc)

50/50 [==============================] - 0s 72us/step
0.6000000238418579


In [117]:
pso.history

[0.6100000143051147,
 0.6299999952316284,
 0.6499999761581421,
 0.6600000262260437,
 0.6600000262260437,
 0.6600000262260437,
 0.6700000166893005,
 0.6700000166893005,
 0.6700000166893005,
 0.6700000166893005,
 0.6700000166893005]

In [112]:
particles_history = [x.history for x in pso.get_particles()]

for particle in particles_history:
    print(particle)
    print()

[0.5600000023841858, 0.6600000262260437, 0.6899999976158142, 0.7099999785423279, 0.7099999785423279, 0.6899999976158142, 0.7200000286102295, 0.7400000095367432, 0.7300000190734863, 0.7400000095367432, 0.7400000095367432]

[0.5600000023841858, 0.6499999761581421, 0.46000000834465027, 0.7099999785423279, 0.7200000286102295, 0.7200000286102295, 0.7300000190734863, 0.7400000095367432, 0.7400000095367432, 0.7400000095367432, 0.7400000095367432]

[0.36000001430511475, 0.6399999856948853, 0.550000011920929, 0.6499999761581421, 0.699999988079071, 0.6700000166893005, 0.7200000286102295, 0.7400000095367432, 0.7400000095367432, 0.7400000095367432, 0.7400000095367432]

[0.20000000298023224, 0.30000001192092896, 0.6299999952316284, 0.699999988079071, 0.6800000071525574, 0.7200000286102295, 0.7300000190734863, 0.7300000190734863, 0.7400000095367432, 0.7400000095367432, 0.7400000095367432]

[0.1599999964237213, 0.6299999952316284, 0.6399999856948853, 0.6800000071525574, 0.6600000262260437, 0.74000000

In [ ]:
from sklearn import datasets
data = datasets.load_breast_cancer()

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data.data, 
                                                    data.target, 
                                                    test_size=0.33)
print(x_train.shape)


from sklearn import preprocessing
print('Pre skaliranja:\n', x_train[0])
scaler = preprocessing.StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)
print('\nPosle skaliranja:\n', x_train[0])


from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
model.add(Dense(units=100, input_dim=x_train.shape[1], activation='relu'))
model.add(Dense(units=2, activation='sigmoid'))
model.summary()


from keras import losses, optimizers
model.compile(optimizer='adam', 
              loss=losses.binary_crossentropy, 
              metrics=['accuracy'])

history = model.fit(x_train, 
                    to_categorical(y_train), 
                    epochs=20, 
                    batch_size=16, 
                    verbose=1, 
                    validation_split=0.3)


from matplotlib import pyplot as plt
%matplotlib inline
epochs = history.epoch
loss = history.history['loss']
validation_loss = history.history['val_loss']
plt.xlabel('epochs')
plt.ylabel('loss')
plt.plot(epochs, loss, c='red', label='training')
plt.plot(epochs, validation_loss, c='orange', label='validation')
plt.legend(loc='best')



In [ ]:
acc = history.history['accuracy']
validation_acc = history.history['val_accuracy']
plt.xlabel('epochs')
plt.ylabel('acc')
plt.plot(epochs, acc, c='red', label='training')
plt.plot(epochs, validation_acc, c='orange', label='validation')
plt.legend(loc='best')


test_loss, test_acc = model.evaluate(x_test, to_categorical(y_test))
print("Test loss:", test_loss) 
print("Test accuracy:", test_acc)

In [ ]:
print(type(model.get_weights()))

print(type(model.get_weights()[0].tolist()))

print(type(model.get_weights()[0][0]))

print(type(model.get_weights()[0][0][0]))

for i in model.get_weights():
    print(i.shape)
    
shapes = [i.shape for i in model.get_weights()]

print("Shapes ", shapes)

low = -2
upp = 2

weights = []
        
for shape in shapes:
    if(len(shape) == 1):
        # wght da dobijemo vrednosti iz (0, upp-low)
        # rec da ih vrati na (low, upp)

        wght = rand(shape[0])*(upp-low)
        rec = np.full(shape, low)

        weights.append(np.add(wght, rec))

    else:
        wght = rand(shape[0], shape[1])*(upp-low)
        rec = np.full(shape, low)

        weights.append(np.add(wght, rec))

print(weights)

In [ ]:
weights = model.get_weights()
print(np.array(weights[0]).shape)
print(np.array(weights[1]).shape)
print(np.array(weights[2]).shape)
print(np.array(weights[3]).shape)

velocity = 0.7 * np.array(weights) + 1.2*0.5*(np.add(np.array(weights), (-0.3)*np.array(weights))) + 1.4*0.7*(np.add(np.array(weights), np.array(weights)))         
print(velocity)